# Trabajar con Compute

In [ ]:
! pip show azure-ai-ml

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## Crear un clúster de cómputo

In [ ]:
from azure.ai.ml.entities import AmlCompute

# Nombre asignado al clúster de cómputo
cpu_compute_target = "aml-cluster"

try:
    # Veamos si el target de cómputo ya existe
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"Ya tienes un clúster llamado {cpu_compute_target}, Lo reutilizaremos como es."
    )

except Exception:
    print("Creación de un nuevo objetivo de cómputo de CPU...")

    # Creemos el objeto Azure ML Compute con los parámetros previstos
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute es el servicio VM bajo demanda
        type="amlcompute",
        # Familia VM
        size="Standard_D1_v2",
        # Nodos mínimos en ejecución cuando no hay trabajo en ejecución
        min_instances=0,
        # Nodos en el clúster
        max_instances=1,
        # ¿Cuántos segundos se ejecutará el nodo después de la terminación del trabajo?
        idle_time_before_scale_down=120,
        # Dedicated o LowPriority.Este último es más barato, pero hay una posibilidad de terminación del trabajo
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

Después de haber creado un clúster de cómputo, solo puede cambiar la configuración para:

- `min_instances`: Número mínimo de nodos
- `max_instances`: Número máximo de nodos
- `idle_time_before_scale_down`: Tiempo de inactividad antes de la escala hacia abajo

Actualmente, el clúster de cómputo `aml-cluster` Solo puede escalar a un máximo de un nodo.Cambiemos eso a dos, para permitir el cálculo paralelo.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=1,
)
ml_client.begin_create_or_update(cluster_scale)

Cuando se actualiza el clúster de cómputo, se puede verificar su configuración imprimiendo sus atributos.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Crear un script para entrenar un modelo

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Ejecutar un trabajo en un clúster de cómputo

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)